In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from TCGA_files import *
from gtex import get_gtex_tissue

In [ ]:
l = 3
#setup = '3000doc_countsweight_quartiles'
#setup='overlap_5000doc_0.25occ_fpkmweight_thr'
setup = 'oversigma_5tissue'
#label = 'disease_type'
label='primary_site'
#directory = '/Users/filippo/Developer/tesi/results/hSBM/%s/topsbm/'%(setup)
directory = '/Users/filippo/Developer/tesi/gtex/hsbm/%s/topsbm/'%setup

In [ ]:
df = pd.read_csv("%s/topsbm_level_%d_topic-dist.csv"%(directory,l))
df.head()

In [ ]:
topic = 6

In [ ]:
searchdf = df.sort_values(by="Topic %d"%topic, ascending=False).loc[:,['doc','Topic %d'%topic]]
searchdf.head()

In [ ]:
#datatotest = queryFiles([f[0]+'.FPKM.txt.gz' for f in searchdf.values[:30] if f[1]>0.1])
#datatotest = queryFiles([f[0] for f in searchdf.values[:10]])
for file in [f[0] for f in searchdf.values[:10]]:
    print(get_gtex_tissue(file))

In [ ]:
#makeTopicPie(datatotest, l, ['primary_site','disease_type'])

## topic distr

In [ ]:
def topic_distr_sample(doc, ax=None):
    if ax == None:
        fig = plt.figure()
        ax = fig.subplots()
    ax.set_title("Topic distribution: %s"%doc)
    labels=[l if df[df['doc']==doc].loc[:,l].values[0]>=0.05 else '' for l in df.columns[2:]]
    patches, texts, autotexts = ax.pie(df[df['doc']==doc].values[0][2:], labels=labels, autopct=lambda p: '%.1f%s'%(p,'%') if p >= 5 else '', textprops={'fontsize':20, 'color':'white', 'wrap':True})
    for t in texts:
                t.set_fontsize(18)
                t.set_wrap(True)
                t.set_color('black')
    plt.show()
    
def topic_distr_isample(idoc, ax=None):
    topic_distr_sample(df[df['i_doc']==idoc]['doc'].values[0],ax)

In [ ]:
for idoc in searchdf.index.values[:5]:
    fig=plt.figure()
    ax=fig.subplots()
    topic_distr_isample(idoc,ax)
    plt.show()
    fig.savefig("topic_distr_%d.png"%idoc)

In [ ]:
topic_distr_isample(153)

In [ ]:
df[df['doc']=='53cd4bb4-5c52-46b6-a37c-c442b536c5e4']

# Clusters

In [ ]:
df_clusters = pd.read_csv("%s/topsbm_level_%d_clusters.csv"%(directory, l), header=[0])
for cluster in df_clusters.columns.values:
    idoc=df[df['doc']==df_clusters.at[0,cluster]]['i_doc'].values[0]
    topic_distr_isample(idoc)